In [ ]:
from evidently.ui.workspace import CloudWorkspace

In [ ]:
ws = CloudWorkspace("", url="https://pr-1885.evidently.dev/")

In [ ]:
org_id = "019396b3-b862-7bdb-95ea-1786c2262724"
project = ws.search_project("Prompts Example", org_id=org_id)[0]

In [ ]:
prompt = ws.prompts.get_or_create_prompt(project.id, "my criteria")
prompt.list_versions()

In [ ]:
criteria = "aaaa"
prompt.bump_version(criteria)

In [ ]:
prompt.list_versions()

In [ ]:
prompt.get_version(1).content

In [ ]:
prompt.get_version("latest").content